<a href="https://colab.research.google.com/github/Oksana0020/Hokku/blob/main/Hokku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic Algorithm Assignment Hokku

## The Problem         **(~30%)**

*   Description of the problem

The goal of this project is to generate a hokku, a very short Japaneese style poem consisting of three lines following the traditional syllables pattern of 5-7-5.

In this implementation, a hokku is generated by selecting words from a predefined vocabulary and arranging them into three lines. The Genetic Algorithm (GA) searches for combinations of words that satisfy structural, grammatical and stylistic rules.

*   Discussion of the suitablity of Genetic Algorithms

*   Complexity of the problem  (Overall marks allocated based on ..)

# The problem and the cost function   **(~20%)**

# The Individual **(~30%)**


*   Chromosone
*   Crossover
*   Mutation

## Discussion and justification on the approaches taken for the above

## Running the algorithm  **(~10%)**

*   Parameter choices
*   Modifications (if any) to run_genetic
*   Rationale for the above

In [ ]:
#  Parameter class here

In [ ]:
# Run Genetic method here

In [ ]:
#  Running of the algorithm with outputs here

In [ ]:
#  If changes to params or reruns of iterations dont overwrite, create more cells and copy code down to show evolution of final solution

### Hokku Generator using Genetic Algorithms

**A hokku** is a very short poem written in three lines following 5–7–5 syllable pattern. In my project, a hokku is generated text built by choosing words from vocabulary and arranging them into 3 lines following syllables pattern 5–7–5.

**A chromosome** is one candidate hokku solution. It is encoded as an array of integers. Each integer corresponds to an index in the vocabulary.The chromosome length is fixed and represents word slots for all three lines of the poem.When decoded, the chromosome maps to words that form the hokku text.

**The cost function** is a scoring rule that measures how bad a candidate hokku is. Lower cost means a better poem, with the ideal goal being to approach zero cost. The cost function is built incrementally and enforces the following rules:

- *Syllable mismatch penalty*
Penalize deviation from the 5–7–5 syllable pattern.

- *Marker rule*
Exactly one marker must be present in the poem.
Later stages prefer the marker at the end of line 1 or line 2.
This enables a later *ending rule* (do not end the poem with a marker or connector).

- *Repetition penalty*
Penalize repeated words within the same poem.

- *Season-word rule*
Require at least one season word per poem (added at a later stage).

- *Grammar rules*
Examples:
1)Preferred word order such as
adj noun verb or adj noun verb adv
2)At least two adverbs within one poem
3)Avoid ending the poem with short connector / preposition words

**Mutation** introduces random variation into an individual chromosome so the Genetic Algorithm can explore new possibilities. During mutation, a word may be replaced by another word from the same category (for example, replacing one noun with a different noun).It allows new word combinations to appear.

**Crossover** combines two parents to produce children: parent1, parent2 are making children by blending genes. Crossover mixes parents’ word choices. Kids  inherit word positions from each parent.Crossover reuses good partial solutions

### Hokku Generator using Genetic Algorithms

**A hokku** is a very short poem written in three lines following 5–7–5 syllable pattern. In my project, a hokku is generated text built by choosing words from vocabulary and arranging them into 3 lines following syllables pattern 5–7–5.

**A chromosome** is one candidate hokku solution. It is encoded as an array of integers. Each integer corresponds to an index in the vocabulary.The chromosome length is fixed and represents word slots for all three lines of the poem.When decoded, the chromosome maps to words that form the hokku text.

**The cost function** is a scoring rule that measures how bad a candidate hokku is. Lower cost means a better poem, with the ideal goal being to approach zero cost. The cost function is built incrementally and enforces the following rules:

- *Syllable mismatch penalty*
Penalize deviation from the 5–7–5 syllable pattern.

- *Marker rule*
Exactly one marker must be present in the poem.
Later stages prefer the marker at the end of line 1 or line 2.
This enables a later *ending rule* (do not end the poem with a marker or connector).

- *Repetition penalty*
Penalize repeated words within the same poem.

- *Season-word rule*
Require at least one season word per poem (added at a later stage).

- *Grammar rules*
Examples:
  - 1)Preferred word order such as
adj noun verb or adj noun verb adv
  - 2)At least two adverbs within one poem
  - 3)Avoid ending the poem with short connector / preposition words

**Mutation** introduces random variation into an individual chromosome so the Genetic Algorithm can explore new possibilities. During mutation, a word may be replaced by another word from the same category (for example, replacing one noun with a different noun, or one season word with another season word).It allows new word combinations to appear.

**Crossover** combines two parents to produce children: parent1, parent2 are making children by blending genes. Crossover mixes parents’ word choices. Kids  inherit word positions from each parent.Crossover reuses solutions

**Example of Hokku**


Through my shanks are thin

I go where flowers blossom,

Yoshino Mountain.


*Basho*

Example is taken from "One Hundred Famous Haiku" by Daniel C. Buchanan
https://thehaikufoundation.org/omeka/files/original/50a61193bc1caf7a3c86b9acc96d4b44.pdf

#Import libraries


*   NumPy is used for efficient chromosome creation and random operations.
*   deepcopy is used later to safely store the best individual without it being overwritten during updates.

In [1]:
import numpy as np
from copy import deepcopy

#Vocabulary
defines vocabulary that Genetic Algorithm can choose from.

Each vocabulary entry includes:

**word**: the actual word or punctuation symbol

**syllable**: number of syllables (used for the 5–7–5 constraint)

**category**: word type (used later for grammar rules and category-aware mutation)

Current vocabulary consists of:
markers (which are punctuation symbols),
nouns, verbs, adjectives, adverbs,
connectors (prepositions/conjunctions in category "other")

*Note: Season words will be added later so cost function can require at least one season word.*

In [2]:
WORD_DATA = [
    # markers
    {"word": "—", "syllable": 0, "category": "marker"},
    {"word": "...", "syllable": 0, "category": "marker"},
    {"word": "!", "syllable": 0, "category": "marker"},
    {"word": "?", "syllable": 0, "category": "marker"},
    {"word": ",", "syllable": 0, "category": "marker"},
    {"word": ".", "syllable": 0, "category": "marker"},
    {"word": ";", "syllable": 0, "category": "marker"},
    {"word": ":", "syllable": 0, "category": "marker"},
    {"word": "@", "syllable": 0, "category": "marker"},
    {"word": "¿", "syllable": 0, "category": "marker"},

    # nouns
    {"word": "pond", "syllable": 1, "category": "noun"},
    {"word": "frog", "syllable": 1, "category": "noun"},
    {"word": "river", "syllable": 2, "category": "noun"},
    {"word": "stone", "syllable": 1, "category": "noun"},
    {"word": "leaf", "syllable": 1, "category": "noun"},
    {"word": "shadow", "syllable": 2, "category": "noun"},
    {"word": "silence", "syllable": 2, "category": "noun"},
    {"word": "moon", "syllable": 1, "category": "noun"},
    {"word": "wind", "syllable": 1, "category": "noun"},
    {"word": "lantern", "syllable": 2, "category": "noun"},

    # verbs
    {"word": "jumps", "syllable": 1, "category": "verb"},
    {"word": "drifts", "syllable": 1, "category": "verb"},
    {"word": "falls", "syllable": 1, "category": "verb"},
    {"word": "whispers", "syllable": 2, "category": "verb"},
    {"word": "shines", "syllable": 1, "category": "verb"},
    {"word": "turns", "syllable": 1, "category": "verb"},
    {"word": "melts", "syllable": 1, "category": "verb"},
    {"word": "settles", "syllable": 2, "category": "verb"},
    {"word": "wanders", "syllable": 2, "category": "verb"},
    {"word": "sleeps", "syllable": 1, "category": "verb"},

    # adjectives
    {"word": "old", "syllable": 1, "category": "adj"},
    {"word": "silent", "syllable": 2, "category": "adj"},
    {"word": "soft", "syllable": 1, "category": "adj"},
    {"word": "bright", "syllable": 1, "category": "adj"},
    {"word": "cold", "syllable": 1, "category": "adj"},
    {"word": "dark", "syllable": 1, "category": "adj"},
    {"word": "misty", "syllable": 2, "category": "adj"},
    {"word": "gentle", "syllable": 2, "category": "adj"},
    {"word": "lonely", "syllable": 2, "category": "adj"},
    {"word": "empty", "syllable": 2, "category": "adj"},

    # connectors
    {"word": "in", "syllable": 1, "category": "other"},
    {"word": "of", "syllable": 1, "category": "other"},
    {"word": "to", "syllable": 1, "category": "other"},
    {"word": "from", "syllable": 1, "category": "other"},
    {"word": "with", "syllable": 1, "category": "other"},
    {"word": "without", "syllable": 2, "category": "other"},
    {"word": "through", "syllable": 1, "category": "other"},
    {"word": "under", "syllable": 2, "category": "other"},
    {"word": "near", "syllable": 1, "category": "other"},
    {"word": "while", "syllable": 1, "category": "other"},

    # adverbs
    {"word": "slowly", "syllable": 2, "category": "adv"},
    {"word": "softly", "syllable": 2, "category": "adv"},
    {"word": "silently", "syllable": 3, "category": "adv"},
    {"word": "brightly", "syllable": 2, "category": "adv"},
    {"word": "deeply", "syllable": 2, "category": "adv"},
    {"word": "lightly", "syllable": 2, "category": "adv"},
    {"word": "suddenly", "syllable": 3, "category": "adv"},
    {"word": "quietly", "syllable": 3, "category": "adv"},
    {"word": "still", "syllable": 1, "category": "adv"},
    {"word": "again", "syllable": 2, "category": "adv"},

    #season words to be added
]

VOCABULARY = [w["word"] for w in WORD_DATA]
SYLLABLE   = {w["word"]: w["syllable"] for w in WORD_DATA}
CATEGORY   = {w["word"]: w["category"] for w in WORD_DATA}
MARKER_SET = {w for w in VOCABULARY if CATEGORY[w] == "marker"}

In [3]:
VOCABULARY[50]

'slowly'

In [4]:
CATEGORY["frog"]

'noun'

In [5]:
SYLLABLE["frog"]

1

##Simple quick validation of vocabulary

In [6]:
from collections import Counter

category_counts = Counter(CATEGORY[w] for w in VOCABULARY)
print("Category counts:", category_counts)
print("Total vocabulary size:", len(VOCABULARY))
print("Markers are the following:", sorted(MARKER_SET))
print("Syllables in 'silently':", SYLLABLE["silently"])
print("Syllables in 'misty':", SYLLABLE["misty"])



Category counts: Counter({'marker': 10, 'noun': 10, 'verb': 10, 'adj': 10, 'other': 10, 'adv': 10})
Total vocabulary size: 60
Markers are the following: ['!', ',', '.', '...', ':', ';', '?', '@', '¿', '—']
Syllables in 'silently': 3
Syllables in 'misty': 2


##Adding chromosome and some helper functions

Decode a chromosome into a 3-line hokku

Count syllables per line

Formating poem nicely for printing

*Note:*

slots_per_line does not mean syllables, it means how many words Genetic Algorithm is allowed to choose per line.

chromosome length is:
number_of_genes = 3 * slots_per_line because a hokku always has 3 lines.

In [7]:
slots_per_line = 4
number_of_genes = 3 * slots_per_line

#Convert a numeric chromosome into three lines of words
def chromosome_to_lines(chromosome, slots_per_line: int):
    words = [VOCABULARY[int(i)] for i in chromosome]
    return [
        words[0:slots_per_line],
        words[slots_per_line:2 * slots_per_line],
        words[2 * slots_per_line:3 * slots_per_line],
    ]

#Calculate total syllables of line
def line_syllables(words):
    return sum(SYLLABLE[w] for w in words)

def format_hokku(lines):
    formatted_lines = []
    for line in lines:
        out = []
        for w in line:
            if w in MARKER_SET:
                if out:
                    out[-1] = out[-1] + w
                else:
                    out.append(w)
            else:
                out.append(w)
        formatted_lines.append(" ".join(out))
    return "\n".join(formatted_lines)

## Checking chromosome decoding using raw integer indices.

Here I verify that chromosome decoding works correctly. Decoding errors could break both the cost function and GA behavior, therefore I included this checkup.

It demonstrates that a chromosome is a list of integers.

Each integer is treated as an index into the vocabulary.

The chromosome is split into 3 lines.

The poem is printed in a readable format.

Syllables are counted per line (this needed for cost function)

In [8]:
chromosome = [
    50, 1, 11, 0,     # line 1
    12, 20, 32, 5,   # line 2
    18, 14, 49, 2,   # line 3
]

lines = chromosome_to_lines(chromosome, slots_per_line)

print("Decoded lines:", lines)
print()
print("Formatted hokku:\n")
print(format_hokku(lines))
print()
print("Syllables per line:", [line_syllables(line) for line in lines])

Decoded lines: [['slowly', '...', 'frog', '—'], ['river', 'jumps', 'soft', '.'], ['wind', 'leaf', 'while', '!']]

Formatted hokku:

slowly... frog—
river jumps soft.
wind leaf while!

Syllables per line: [3, 4, 3]


# Cost function version 1 (penalty for syllable mismatch)

This is the first version of the cost function.
At this stage, the only rule is the hokku syllable structure: 5–7–5.

How function works:

Decode the chromosome into 3 lines

Compute syllables per line

Add a penalty equal to the absolute difference from target syllables

This means:

Lower cost = better poem

Cost = 0 means the syllable pattern matches perfectly (5–7–5)


In [9]:
def hokku_cost_v1(chromosome, slots_per_line: int) -> float:
    target = [5, 7, 5]

    lines = chromosome_to_lines(chromosome, slots_per_line)
    syllables = [line_syllables(line) for line in lines]

    cost = 0.0
    for i in range(3):
        cost += abs(syllables[i] - target[i])

    return float(cost)


## Test cost function version 1 on manual chromosome example

Checking that cost function is working correctly.

It prints:
*   The syllable totals for each of the three lines
*   The resulting cost value

If the poem follows pattern exactly, the perfect hokku (5–7–5) would give cost = 0.
Otherwise, the cost measures how far away the poem is from 5–7–5.

In [10]:
syllables = [line_syllables(line) for line in lines]
print("Syllables per line:", syllables)
print("Cost is:", hokku_cost_v1(chromosome, slots_per_line))


Syllables per line: [3, 4, 3]
Cost is: 7.0


In this example, decoded poem produces Syllables per line: [3, 4, 3], which is far from the target [5, 7, 5].

Therefore, the total deviation is:

|3 − 5| = 2

|4 − 7| = 3

|3 − 5| = 2

Total cost = 2 + 3 + 2 = 7

So cost function correctly measures syllable mismatch.

Before implementing Genetic Algorithm, I plan to evaluate initial cost function
on many randomly generated chromosomes.

When Genetic Algorithm works correctly, it should produce better results than this random function.

##Randomly generated chromosome

A chromosome is generated randomly, where each gene represents a random index from the vocabulary.

This process is repeated multiple times and cost of each chromosome is calculated using cost function.

The best chromosome found during random sampling is recorded (the best one is the one with the lowest cost) .

Random search provides a baseline benchmark that represents the minimum expected performance without optimization.

My expectation is that Genetic Algorithm should consistently produce better solutions than random guessing by using selection, crossover and mutation.

In [11]:
import random

def random_chromosome(number_of_genes: int):
    chromosome = []
    for _ in range(number_of_genes):
        gene = random.randint(0, len(VOCABULARY) - 1)
        chromosome.append(gene)
    return chromosome


best_cost = float("inf") #possible infinity
best_chromosome = None
trials = 1000

for _ in range(trials):
    chrom = random_chromosome(number_of_genes)
    c = hokku_cost_v1(chrom, slots_per_line)

    if c < best_cost:
        best_cost = c
        best_chromosome = chrom


best_lines = chromosome_to_lines(best_chromosome, slots_per_line)

print("Trials:", trials)
print("Best cost found:", best_cost)
print("Best random hokku:\n")
print("====================")
print(format_hokku(best_lines))
print("====================")
print("Syllables per line:", [line_syllables(line) for line in best_lines])

Trials: 1000
Best cost found: 0.0
Best random hokku:

? through brightly whispers
from brightly suddenly turns
river cold pond stone
Syllables per line: [5, 7, 5]


Random search rarely produces poems that satisfy 5–7–5 pattern because the probability of hitting correct syllable totals by chance is very low. Genetic Algorithm should outperform random search by reducing cost through crossover and mutation.

#Genetic Algorithm version1

Implement a Genetic Algorithm version1 to optimize hokku syllable structure 5-7-5


- an individual is one candidate hokku solution,
- chromosome is a fixed-length array of integer word indices in vocabulary,
- cost is syllable mismatch penalty only for now,
- mutation randomly replaces some indices,
- crossover combines two parents at a random cut point.

Goal: reach cost = 0 (perfect 5–7–5), faster than random sampling.

##Quick check of vocabulary size

Checking vocabulary size and determing valid range of gene values:

Minimum gene value = 0

Maximum gene value will be = len(VOCABULARY) - 1

In [12]:
len(VOCABULARY)

60

##Define GA problem class

This class stores problem configuration in one place:

*   slots_per_line: how many word slots each line contains
*   number_of_genes: total word slots in the poem (3 lines × slots per line)
*   min_gene_value and max_gene_value: valid index range for vocabulary lookup
*   cost_function: references hokku_cost_v1 so the GA can evaluate individuals

Such design makes it easy to later switch cost function versions (v2, v3, …etc) without rewriting GA.

In [13]:
class problem:
    def __init__(self, slots_per_line: int = 4):
        self.slots_per_line = int(slots_per_line)
        self.number_of_genes = 3 * self.slots_per_line
        self.min_gene_value = 0
        self.max_gene_value = len(VOCABULARY) - 1

        self.cost_function = lambda chrom: hokku_cost_v1(chrom, self.slots_per_line)

##Define GA hyperparameters

This class stores main parameters that control GA behaviour:

*   population_size: number of individuals per generation
*   generations: maximum number of iterations
*   child_rate: proportion of new children created per generation
*   mutation_rate: probability that any gene will be replaced randomly
These values affect convergence speed, diversity, and runtime.

In [14]:
class Parameters:
    def __init__(self):
        self.population_size = 1000
        self.generations = 300
        self.child_rate = 0.6
        self.mutation_rate = 0.15 #(15% chance to mutate, 85% chance to stay the same)

##Define an Individual Representation, Crossover and Mutation

**Individual** class represents one candidate solution in Genetic Algorithm.

Each individual contains a **chromosome**, which is implemented as NumPy array of integers. Each integer represents an index into the vocabulary and therefore corresponds to a specific word in the hokku.

**Chromosome**

Fixed-length array of integers

Each gene stores a valid vocabulary index

Represents one complete hokku candidate

**Single-Point Crossover**

Crossover is implemented using a single-point method.

A random cut point is selected along the chromosome.

The child inherits the first part of its chromosome from Parent A.

The remaining part is inherited from Parent B.

This allows useful partial solutions from different parents to be combined.

**Mutation**

Mutation introduces random variation to maintain genetic diversity.

For each gene, a random number is generated.

If it is below mutation_rate, the gene is replaced with a new random valid index.

This version uses uniform random mutation, where any gene can be replaced by any valid vocabulary index.

In later versions I will upgrade mutation to be category-aware (noun → noun, verb → verb).

In [15]:
class Individual:
    def __init__(self, prob: problem):
        self.chromosome = np.random.randint(
            prob.min_gene_value,
            prob.max_gene_value + 1,
            size=prob.number_of_genes,
        )
        self.cost = prob.cost_function(self.chromosome)

    def crossover(self, other: "Individual") -> np.ndarray:
        n = len(self.chromosome)
        point = np.random.randint(1, n)
        return np.concatenate([self.chromosome[:point], other.chromosome[point:]]).astype(int)

    def mutate(self, prob: problem, mutation_rate: float):
        for i in range(len(self.chromosome)):
            if np.random.rand() < mutation_rate:
                self.chromosome[i] = np.random.randint(prob.min_gene_value, prob.max_gene_value + 1)

###Parent selection helper (random selection)

Simple parent selection method randomly pick two different individuals from the current population. If the same index is selected twice, the function retries.

The GA needs a method to select parents for crossover. This version uses random selection (simple baseline)

Later improvements can use other ways to choose parents for reproduction: tournament selection or roulette-wheel selection.

*(I need to investigate different ways of choosing parent)*

https://www.baeldung.com/cs/ga-tournament-selection

https://www.baeldung.com/cs/genetic-algorithms-roulette-selection

In [16]:
def choose_two_parents(population: list[Individual]):
    i1 = np.random.randint(0, len(population))
    i2 = np.random.randint(0, len(population))
    if i1 == i2:
        return choose_two_parents(population)
    return population[i1], population[i2]

##Genetic Algorithm runner (main loop)

This cell implements the full Genetic Algorithm loop.

It initializes population randomly, evaluate and sort individuals by cost (lower = better) and tracks the best solution found so far.

For each generation:
*   Create a set of children.
Each child is produced by selecting two parents, applying crossover, applying mutation, computing cost
*   Add children to population
*   Sort again and keep only the top individuals (survival of the fittest)
*   Print progress every 20 generations
*   Stop early if a perfect solution (cost = 0) is found

This design ensures the population improves over time while maintaining a fixed population size.

In [17]:
def run_genetic(prob: problem, params: Parameters):
    population = [Individual(prob) for _ in range(params.population_size)]
    population.sort(key=lambda ind: ind.cost)

    best = deepcopy(population[0])
    children_per_generation = int(params.population_size * params.child_rate)

    for gen in range(params.generations):
        children = []

        while len(children) < children_per_generation:
            parent1, parent2 = choose_two_parents(population)

            child_chrom = parent1.crossover(parent2)

            child = Individual(prob)
            child.chromosome = child_chrom
            child.mutate(prob, params.mutation_rate)
            child.cost = prob.cost_function(child.chromosome)

            children.append(child)

        population.extend(children)
        population.sort(key=lambda ind: ind.cost)
        population = population[:params.population_size]

        if population[0].cost < best.cost:
            best = deepcopy(population[0])

        if gen % 20 == 0:
            print("Generation:", gen, "Best cost:", best.cost)

        if best.cost == 0:
            break

    return best

##Run GA version 1 and print the best hokku found

Running GA using problem definition and parameter settings:

decodes the best chromosome into 3 lines

prints the poem in formatted text

prints the syllable totals to verify the 5–7–5 structure

prints the final cost (ideally 0)

In [18]:
prob = problem(slots_per_line=slots_per_line)
params = Parameters()

best = run_genetic(prob, params)
best_lines = chromosome_to_lines(best.chromosome, slots_per_line)

print("\nBest GA cost:", best.cost)
print("Best GA hokku:\n")
print("====================")
print(format_hokku(best_lines))
print("====================")
print("Syllables per line:", [line_syllables(line) for line in best_lines])

Generation: 0 Best cost: 0.0

Best GA cost: 0.0
Best GA hokku:

... lantern shadow of
lonely old silently through
soft whispers near old
Syllables per line: [5, 7, 5]


#Cost Function (version 2) syllables and marker rule

Cost function is upgraded here by adding a marker rule on top of the 5–7–5 syllable constraint.

New rule:

The hokku must contain exactly one marker (punctuation symbol).

If there are no markers, add a large penalty.

If there are multiple markers, add a penalty for each extra marker.

This new rule encourages poems to have clear pause or emphasis point.


In traditional hokku, a cutting word often appears at the end of one phrase to create a pause, contrast or shift in tone. In English translations, punctuation is used to show this effect.

Wikipedia (2026) Hokku. Available at: https://en.wikipedia.org/wiki/Hokku
 (Accessed: 19 February 2026)


Cost Function (version 2) prepares for future rules, such as preferring the marker at the end of line 1 or 2 becuase  emphasis point has to be somewhere inside the poem changing tonallity or showing contradiction.

A later improvement can include rule to prevent ending the poem with a marker (no marker at end of line 3).

In [19]:
def hokku_cost_v2(chromosome, slots_per_line: int) -> float:
    target = [5, 7, 5]

    lines = chromosome_to_lines(chromosome, slots_per_line)
    syllables = [line_syllables(line) for line in lines]

    # syllable penalty
    cost = 0.0
    for i in range(3):
        cost += abs(syllables[i] - target[i])

    # marker penalty
    all_words = [w for line in lines for w in line]
    marker_count = sum(1 for w in all_words if w in MARKER_SET)

    if marker_count == 0:
        cost += 10.0
    elif marker_count > 1:
        cost += 5.0 * (marker_count - 1)

    return float(cost)

In [ ]:
best = run_genetic(prob, params)

Generation: 0 Best cost: 0.0


##Helper function to count markers in a chromosome

I use it for debugging and reporting.
It counts how many markers appear in a chromosome after decoding it into words.

In [20]:
def marker_count_in_chromosome(chromosome, slots_per_line: int) -> int:
    lines = chromosome_to_lines(chromosome, slots_per_line)
    all_words = [w for line in lines for w in line]
    return sum(1 for w in all_words if w in MARKER_SET)


##Test Cost function version 2 (syllables rule, marker rule) on manual chromosome example

Cost function version 2 still measures syllable mismatch,
adds a penalty when the poem has zero markers or more than one marker.

Validating that version 2 works correctly by printing:

*   the formatted poem
*   syllables per line
*   marker count
*   the final v2 cost

In [21]:
lines = chromosome_to_lines(chromosome, slots_per_line)

print("Formatted hokku:\n")
print(format_hokku(lines))
print()

print("Syllables per line:", [line_syllables(line) for line in lines])
print("Marker count:", marker_count_in_chromosome(chromosome, slots_per_line))
print("Cost v2:", hokku_cost_v2(chromosome, slots_per_line))


Formatted hokku:

slowly... frog—
river jumps soft.
wind leaf while!

Syllables per line: [3, 4, 3]
Marker count: 4
Cost v2: 22.0


##Additional tests: chromosomes with 0 markers and 2 markers

Tests check extreme cases and confirm that marker penalties behave as intended.

Test cases:

**No-marker chromosome** should trigger missing marker penalty (+10)

**Two-marker chromosome** should trigger extra marker penalty (+5 per extra marker)

In [22]:
# Chromosome with no markers
no_marker_chrom = [10] * number_of_genes
print("No-marker marker count:", marker_count_in_chromosome(no_marker_chrom, slots_per_line))
print("No-marker cost v2:", hokku_cost_v2(no_marker_chrom, slots_per_line))
print()

# Chromosome with 2 markers
two_marker_chrom = [0, 1] + [10] * (number_of_genes - 2)
print("Two-marker marker count:", marker_count_in_chromosome(two_marker_chrom, slots_per_line))
print("Two-marker cost v2:", hokku_cost_v2(two_marker_chrom, slots_per_line))


No-marker marker count: 0
No-marker cost v2: 15.0

Two-marker marker count: 2
Two-marker cost v2: 12.0


## Genetic Algorithm version 2 (optimize syllables and marker rule)

Using Cost Function v2 instead of v1.

GA should now generate poems having:
- 5–7–5 syllables,
- exactly one marker in the poem.

This demonstrates benefits of modular design, I can improve behaviour of GA simply by swapping the cost function.

In [23]:
class problem:
    def __init__(self, slots_per_line: int = 4):
        self.slots_per_line = int(slots_per_line)
        self.number_of_genes = 3 * self.slots_per_line
        self.min_gene_value = 0
        self.max_gene_value = len(VOCABULARY) - 1

        self.cost_function = lambda chrom: hokku_cost_v2(chrom, self.slots_per_line)


##Running GA Version 2 and display the best hokku found

In [24]:
prob = problem(slots_per_line=slots_per_line)
params = Parameters()

best = run_genetic(prob, params)
best_lines = chromosome_to_lines(best.chromosome, slots_per_line)

print("\nBest GA version 2 cost:", best.cost)
print("Marker count:", marker_count_in_chromosome(best.chromosome, slots_per_line))
print("Best GA version 2 hokku:\n")
print("====================")
print(format_hokku(best_lines))
print("====================")
print("Syllables per line:", [line_syllables(line) for line in best_lines])


Generation: 0 Best cost: 0.0

Best GA version 2 cost: 0.0
Marker count: 1
Best GA version 2 hokku:

old settles soft cold
under silence to lightly
@ jumps settles whispers
Syllables per line: [5, 7, 5]


## Cost Function version 3. Marker position preference (end of line 1 or 2)

In next step I plan to penalize poems where the marker is placed in line 3
or not positioned at the end of line 1 or line 2.

The idea is to encourage a traditional structural break in poem which in English translations is reached with punctuation marks.

In [29]:
def hokku_cost_v3(chromosome, slots_per_line: int) -> float:
    target = [5, 7, 5]

    lines = chromosome_to_lines(chromosome, slots_per_line)
    syllables = [line_syllables(line) for line in lines]
    cost = 0.0
    for i in range(3):
        cost += abs(syllables[i] - target[i])

    all_words = [w for line in lines for w in line]
    marker_positions = [(i, j)
                        for i, line in enumerate(lines)
                        for j, w in enumerate(line)
                        if w in MARKER_SET]

    marker_count = len(marker_positions)
    if marker_count == 0:
        cost += 10.0
    elif marker_count > 1:
        cost += 5.0 * (marker_count - 1)

    # position preference for markers
    if marker_count == 1:
        line_index, word_index = marker_positions[0]

        # penalize if marker is in line 3
        if line_index == 2:
            cost += 6.0

        # penalize if marker is not at end of line 1 or2
        if line_index in [0, 1] and word_index != slots_per_line - 1:
            cost += 4.0

    return float(cost)

Checkup of cost function version 3

In [30]:
lines = chromosome_to_lines(best.chromosome, slots_per_line)

print("Formatted hokku:\n")
print(format_hokku(lines))
print()
print("Syllables per line:", [line_syllables(line) for line in lines])
print("Marker count:", marker_count_in_chromosome(best.chromosome, slots_per_line))
print("Cost v3:", hokku_cost_v3(best.chromosome, slots_per_line))

Formatted hokku:

through deeply without,
quietly again leaf drifts
near deeply wind to

Syllables per line: [5, 7, 5]
Marker count: 1
Cost v3: 0.0


Updating GA to run new version3 of cost function

In [31]:
class problem:
    def __init__(self, slots_per_line: int = 4):
        self.slots_per_line = int(slots_per_line)
        self.number_of_genes = 3 * self.slots_per_line
        self.min_gene_value = 0
        self.max_gene_value = len(VOCABULARY) - 1
        self.cost_function = lambda chrom: hokku_cost_v3(chrom, self.slots_per_line)

Run GA again

In [32]:
prob = problem(slots_per_line=slots_per_line)
params = Parameters()
best = run_genetic(prob, params)
best_lines = chromosome_to_lines(best.chromosome, slots_per_line)

print("\nBest GA v3 cost:", best.cost)
print("Best GA v3 hokku:\n")
print("====================")
print(format_hokku(best_lines))
print("====================")
print("Syllables per line:", [line_syllables(line) for line in best_lines])

Generation: 0 Best cost: 1.0

Best GA v3 cost: 0.0
Best GA v3 hokku:

gentle under stone;
with lonely shadow brightly
while lonely stone wind
Syllables per line: [5, 7, 5]


## Results and conclusions    **(~10%)**